In [1]:
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib

In [2]:
data=pd.read_csv(r"C:\Users\gagan\Downloads\Data\Winter 23-24\Capstone\fraud-transactions-detection\JPMorgan\SupervisedFraudData.csv")
data.head(2)

,Time_step,Transaction_Id,Sender_Id,Sender_Account,Sender_Country,Sender_Sector,Sender_lob,Bene_Id,Bene_Account,Bene_Country,USD_amount,Label,Transaction_Type
0,2022-03-15 10:24:00,EXCHANGE-10115,JPMC-CLIENT-10098,ACCOUNT-10108,USA,35537,CCB,Unknown,Unknown,Unknown,558.43,0,WITHDRAWAL
1,2022-03-15 10:24:00,QUICK-PAYMENT-10116,JPMC-CLIENT-10098,ACCOUNT-10109,USA,15287,CCB,CLIENT-10100,ACCOUNT-10106,CANADA,622.78,0,QUICK-PAYMENT


## 1. Data Preprocessing
#### Handle missing values, outliers, encode categorical variables, scale numerical features

In [3]:
data.isnull().sum()

Time_step           0
Transaction_Id      0
Sender_Id           0
Sender_Account      0
Sender_Country      0
Sender_Sector       0
Sender_lob          0
Bene_Id             0
Bene_Account        0
Bene_Country        0
USD_amount          0
Label               0
Transaction_Type    0
dtype: int64

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1498177 entries, 0 to 1498176
Data columns (total 13 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   Time_step         1498177 non-null  object 
 1   Transaction_Id    1498177 non-null  object 
 2   Sender_Id         1498177 non-null  object 
 3   Sender_Account    1498177 non-null  object 
 4   Sender_Country    1498177 non-null  object 
 5   Sender_Sector     1498177 non-null  int64  
 6   Sender_lob        1498177 non-null  object 
 7   Bene_Id           1498177 non-null  object 
 8   Bene_Account      1498177 non-null  object 
 9   Bene_Country      1498177 non-null  object 
 10  USD_amount        1498177 non-null  float64
 11  Label             1498177 non-null  int64  
 12  Transaction_Type  1498177 non-null  object 
dtypes: float64(1), int64(2), object(10)
memory usage: 148.6+ MB


In [5]:
data['Sender_lob'].value_counts()

Sender_lob
CCB        1279291
Unknown     218886
Name: count, dtype: int64

In [6]:
data_encoded = pd.get_dummies(data, columns=['Sender_Country', 'Bene_Country', 'Transaction_Type'])


In [7]:
label_encoder = LabelEncoder()
data_encoded['Transaction_Id'] = label_encoder.fit_transform(data_encoded['Transaction_Id'])
data_encoded['Sender_Id'] = label_encoder.fit_transform(data_encoded['Sender_Id'])
data_encoded['Sender_Account'] = label_encoder.fit_transform(data_encoded['Sender_Account'])
data_encoded['Bene_Id'] = label_encoder.fit_transform(data_encoded['Bene_Id'])
data_encoded['Bene_Account'] = label_encoder.fit_transform(data_encoded['Bene_Account'])

In [8]:
data_encoded.drop('Sender_lob',inplace=True,axis=1)
data_encoded.drop('Time_step',inplace=True,axis=1)

In [9]:
# 2. Split the Data
Feature = data_encoded.drop(columns=['Label'])  # Features
Target = data_encoded['Label']  # Target variable
Feature_train, Feature_test, Target_train, Target_test = train_test_split(Feature, Target, test_size=0.2, random_state=42)

In [10]:
# 3. Train the Isolation Forest Model
model = IsolationForest(n_estimators=100, contamination=0.01, random_state=42)
model.fit(Feature_train)

IsolationForest(contamination=0.01, random_state=42)

In [11]:
# 4. Evaluate the Model
Target_pred_train = model.predict(Feature_train)
Target_pred_test = model.predict(Feature_test)

print("Classification Report for Training Set:")
print(classification_report(Target_train, Target_pred_train))

print("Classification Report for Test Set:")
print(classification_report(Target_test, Target_pred_test))

Classification Report for Training Set:


c:\Users\gagan\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\gagan\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\gagan\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00         0
           0       0.00      0.00      0.00   1173799
           1       0.02      0.97      0.04     24742

    accuracy                           0.02   1198541
   macro avg       0.01      0.32      0.01   1198541
weighted avg       0.00      0.02      0.00   1198541

Classification Report for Test Set:
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00         0
           0       0.00      0.00      0.00    293559
           1       0.02      0.97      0.04      6077

    accuracy                           0.02    299636
   macro avg       0.01      0.32      0.01    299636
weighted avg       0.00      0.02      0.00    299636



c:\Users\gagan\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\gagan\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
